In [1]:
# --- 1. SETUP AND ENVIRONMENT ---
from google.colab import drive
import os

print("--- Mounting Google Drive and Unpacking Project ---")
drive.mount('/content/drive')

# IMPORTANT: Update this zip file name to match the one you uploaded
!unzip -o /content/drive/MyDrive/powershell-sentinel-main-v4.zip -d /content/
%cd /content/powershell-sentinel-main

print("\n--- Authenticating with Hugging Face ---")
# This is critical for downloading the Llama 3 model
!huggingface-cli login
# Paste your Hugging Face 'read' access token when prompted

print("\n--- Installing and Rebuilding Dependencies ---")
# This is the definitive installation procedure. It uninstalls potentially
# conflicting versions and then installs the exact, known-compatible versions.
!pip uninstall bitsandbytes triton -y -q
!pip install -U "transformers==4.41.2" "datasets==2.19.1" "accelerate==0.30.1" "peft==0.10.0" "trl==0.8.6" "bitsandbytes==0.43.1" "triton==2.3.1" -q

print("\n\n✅ ✅ ✅ MASTER SETUP COMPLETE. You are ready to train. ✅ ✅ ✅")

--- Mounting Google Drive and Unpacking Project ---
Mounted at /content/drive
Archive:  /content/drive/MyDrive/powershell-sentinel-main-v4.zip
4a9cdf9a86521f73bb42bf5c10ab75e1719247b7
   creating: /content/powershell-sentinel-main/
  inflating: /content/powershell-sentinel-main/.gitignore  
  inflating: /content/powershell-sentinel-main/README.md  
  inflating: /content/powershell-sentinel-main/_test_engine.py  
  inflating: /content/powershell-sentinel-main/analyze_audit_log.py  
   creating: /content/powershell-sentinel-main/data/
   creating: /content/powershell-sentinel-main/data/generated/
  inflating: /content/powershell-sentinel-main/data/generated/training_data_v0.json  
  inflating: /content/powershell-sentinel-main/data/generated/training_data_v0_clean.json  
   creating: /content/powershell-sentinel-main/data/interim/
   creating: /content/powershell-sentinel-main/data/interim/curating_logs/
  inflating: /content/powershell-sentinel-main/data/interim/curating_logs/uncurated_

In [2]:
# --- DISASTER RECOVERY: RESTORE TRAINED MODELS ---
print("--- Restoring trained models from Google Drive backups ---")
!unzip -o /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/v1_flat_model_adapters.zip -d /content/powershell-sentinel-main/
# !unzip -o /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/v2_flat_model_adapters.zip -d /content/powershell-sentinel-main/
print("✅ Models restored successfully.")

--- Restoring trained models from Google Drive backups ---
Archive:  /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/v1_flat_model_adapters.zip
   creating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/
   creating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/checkpoint-500/
  inflating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/checkpoint-500/tokenizer.json  
  inflating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/checkpoint-500/adapter_config.json  
  inflating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/checkpoint-500/optimizer.pt  
  inflating: /content/powershell-sentinel-main/content/powershell-sentinel-main/models/v1_flat_model/checkpoint-500/trainer_state.json  
  inflating: /content/powershell-sentinel-main/content/powershell-sentinel-main/model

In [3]:
# --- TASK 2.2 (PART A): PREPARE THE V2 DATASETS (Final Version) ---
import os

print("--- Preparing a clean, leak-free dataset split for the v2 data ---")

# Define all paths for the v2 data workflow
v2_source_data = 'data/migration_test_sandbox/training_data_v2.json'
v2_clean_data = 'data/migration_test_sandbox/training_data_v2_clean.json'
v2_nested_train_out = 'data/sets/v2_provisional/training_set.json'
v2_nested_test_out = 'data/sets/v2_provisional/test_set.json'
v2_flat_train_out = 'data/migration_test_sandbox/training_set_v2_flat.json'
v2_flat_test_out = 'data/migration_test_sandbox/test_set_v2_flat.json'

# --- Step 1: De-duplicate the V2 Source Data ---
# Use your existing script with custom arguments to create a clean v2 dataset.
print("\n[1/3] De-duplicating the provisional v2 dataset...")
!python -m scripts.deduplicate_dataset --input {v2_source_data} --output {v2_clean_data}

# --- Step 2: Partition the CLEAN V2 Dataset ---
# Use your legacy-aware script with custom arguments and a new seed.
print("\n[2/3] Partitioning the de-duplicated v2 dataset...")
!python -m scripts.partition_dataset_legacy \
    --input {v2_clean_data} \
    --train-out {v2_nested_train_out} \
    --test-out {v2_nested_test_out} \
    --mini-train-out data/sets/v2_provisional/mini_train.json \
    --mini-val-out data/sets/v2_provisional/mini_val.json \
    --seed 99

# --- Step 3: Migrate the new V2 Partitions to Flat Format ---
# This step remains the same.
print("\n[3/3] Migrating the new v2 partitions to the flat format...")
!python scripts/migrate_dataset_to_flat.py \
    {v2_nested_train_out} \
    {v2_flat_train_out}

!python scripts/migrate_dataset_to_flat.py \
    {v2_nested_test_out} \
    {v2_flat_test_out}

print("\n\n✅ ✅ ✅ V2 DATASET DE-DUPLICATED, PARTITIONED, AND FLATTENED. Ready for training. ✅ ✅ ✅")

--- Preparing a clean, leak-free dataset split for the v2 data ---

[1/3] De-duplicating the provisional v2 dataset...
--- De-duplicating Dataset ---
Loading raw data from: data/migration_test_sandbox/training_data_v2.json
Saving clean, de-duplicated data to: data/migration_test_sandbox/training_data_v2_clean.json

--- De-duplication Report ---
Original Pair Count: 9,520
Clean (Unique) Pair Count: 9,284
Removed Redundant Pairs: 236
-------------------------------

[2/3] Partitioning the de-duplicated v2 dataset...
--- Starting MLOps Data Partitioning (Legacy) ---
Loading and validating clean dataset from: data/migration_test_sandbox/training_data_v2_clean.json
Successfully validated 9284 clean training pairs.
Shuffling the dataset with random seed: 99...
Split complete: 8355 training samples, 929 test samples.
Mini-split complete: 900 mini-train samples, 100 validation samples.

Saved training set to: data/sets/v2_provisional/training_set.json
Saved test set to: data/sets/v2_provisiona

In [4]:
# --- TASK 2.2 (PART B): TRAIN THE V2-FLAT PROVISIONAL MODEL ---
import json
import os

print("\n--- Preparing for v2-flat (provisional) model training run ---")

# Define paths to our NEWLY CREATED, clean, and flattened v2 datasets
train_path_v2_flat = 'data/migration_test_sandbox/training_set_v2_flat.json'
test_path_for_v2_preflight = 'data/migration_test_sandbox/test_set_v2_flat.json'

with open(train_path_v2_flat, 'r') as f:
    num_samples = len(json.load(f))

# Train for 2 full epochs for a fair comparison
effective_batch_size = 4
steps_per_epoch = num_samples // effective_batch_size
max_steps_for_2_epochs = steps_per_epoch * 2

print(f"\nFound {num_samples} samples in the v2 provisional flattened training set.")
print(f"Training for 2 epochs will take exactly {max_steps_for_2_epochs} steps.")

# Define unique paths for this model
output_dir_v2 = "models/v2_flat_model"
drive_backup_path_v2 = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/v2_flat_model_adapters.zip"

print(f"\n--- Starting training for the v2-flat model. Output will be saved to {output_dir_v2} ---")

!python -m powershell_sentinel.train \
    --model_name meta-llama/Meta-Llama-3-8B-Instruct \
    --train_dataset {train_path_v2_flat} \
    --preflight_train_dataset {train_path_v2_flat} \
    --test_dataset {test_path_for_v2_preflight} \
    --output_dir {output_dir_v2} \
    --learning_rate 2e-5 \
    --lora_rank 16 \
    --max_steps {max_steps_for_2_epochs}

print(f"\n--- Backing up v2-flat model adapters to Google Drive ---")
!mkdir -p /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable
!zip -r {drive_backup_path_v2} /content/powershell-sentinel-main/{output_dir_v2}

print(f"\n\n✅ ✅ ✅ V2-FLAT PROVISIONAL MODEL TRAINING COMPLETE AND BACKED UP. ✅ ✅ ✅")


--- Preparing for v2-flat (provisional) model training run ---

Found 8355 samples in the v2 provisional flattened training set.
Training for 2 epochs will take exactly 4176 steps.

--- Starting training for the v2-flat model. Output will be saved to models/v2_flat_model ---
2025-09-06 07:52:31.190959: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-06 07:52:31.208952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757145151.230705    6297 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757145151.237381    6297 cuda_b

In [ ]:
# --- TASK 2.3: THE FINAL EVALUATION GAUNTLET (with Persistence) ---

print("--- Preparing for the final, comprehensive evaluation run ---")

# Define all paths
legacy_test_set = 'data/migration_test_sandbox/test_set_v0_flat.json'
diversity_test_set = 'data/migration_test_sandbox/test_set_v2_flat.json'
model_path_v1 = "models/v1_flat_model/final_checkpoint"
model_path_v2 = "models/v2_flat_model/final_checkpoint"
audit_log_path_v2 = 'data/migration_test_sandbox/audit_log.jsonl'
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Define paths for the output report files
results_dir = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports"
report_path_1 = f"{results_dir}/report_1_v1_on_legacy.txt"
report_path_2 = f"{results_dir}/report_2_v1_on_diversity.txt"
report_path_3 = f"{results_dir}/report_3_v2_on_diversity.txt"

# Create the results directory on Google Drive
!mkdir -p {results_dir}
print(f"✅ Evaluation reports will be saved to: {results_dir}")


# --- RUN 1: V1 Model on LEGACY Test Set (Showcasing Competence) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 1/3: V1 Model on LEGACY v1 Test Set (Home Turf) ---\n" + "="*80 + "\n")
# THE FIX: Add `> {report_path_1}` to redirect output to a file
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v1} \
    --base_model_path {base_model_name} \
    --test_set_path {legacy_test_set} > {report_path_1}
# Also print the saved report to the screen
!cat {report_path_1}
print(f"✅ Report for Evaluation 1/3 saved to {report_path_1}")


# --- RUN 2: V1 Model on DIVERSITY Test Set (Revealing Brittleness) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 2/3: V1 Model on DIVERSITY v2 Test Set (Away Game) ---\n" + "="*80 + "\n")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v1} \
    --base_model_path {base_model_name} \
    --test_set_path {diversity_test_set} > {report_path_2}
!cat {report_path_2}
print(f"✅ Report for Evaluation 2/3 saved to {report_path_2}")


# --- RUN 3: V2 Model on DIVERSITY Test Set (Proving Superiority & Breakdown) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 3/3: V2 Model on DIVERSITY v2 Test Set (Proving Superiority) ---\n" + "="*80 + "\n")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v2} \
    --base_model_path {base_model_name} \
    --test_set_path {diversity_test_set} \
    --obfuscation-breakdown \
    --audit-log-path {audit_log_path_v2} > {report_path_3}
!cat {report_path_3}
print(f"✅ Report for Evaluation 3/3 saved to {report_path_3}")


print("\n\n✅ ✅ ✅ COMPREHENSIVE EVALUATION GAUNTLET COMPLETE. All reports are saved to Google Drive. ✅ ✅ ✅")

--- Preparing for the final, comprehensive evaluation run ---
✅ Evaluation reports will be saved to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports


--- EVALUATION 1/3: V1 Model on LEGACY v1 Test Set (Home Turf) ---

2025-09-05 07:01:51.431335: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 07:01:51.449965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757055711.472266   51062 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757055711.479139   51062 cuda_blas.cc:1407] Unable to reg

In [ ]:
# --- DEFINITIVE SMOKE TEST FOR BREAKDOWN ANALYSIS (Final Version) ---
import json
import os
import random

print("--- Preparing a 'Golden' smoke test for the breakdown-by-primitive analysis ---")

# ... (all path definitions and data prep logic are correct) ...
# Define all necessary paths
full_test_set_path = 'data/migration_test_sandbox/test_set_v2_flat.json'
golden_smoke_test_path = 'data/migration_test_sandbox/golden_smoke_test_v2.json'
model_path_v2 = "models/v2_flat_model/final_checkpoint"
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# --- Step 1: Programmatically build the golden test set ---
print(f"\n[1/2] Loading full test set from {full_test_set_path} to find representative samples...")
with open(full_test_set_path, 'r', encoding='utf-8') as f: full_data = json.load(f)
seen_primitives = set()
golden_test_set = []
random.shuffle(full_data)
for item in full_data:
    deobfuscated_cmd = item.get('response', {}).get('deobfuscated_command')
    if deobfuscated_cmd and deobfuscated_cmd not in seen_primitives:
        golden_test_set.append(item)
        seen_primitives.add(deobfuscated_cmd)
    if len(golden_test_set) >= 5: break
with open(golden_smoke_test_path, 'w') as f: json.dump(golden_test_set, f, indent=2)
print(f"✅ Created a golden smoke test file with {len(golden_test_set)} diverse primitives at: {golden_smoke_test_path}")

# --- Step 2: Execute the smoke test ---
print("\n[2/2] Executing smoke test evaluation run...")
# --- THE CRITICAL FIX IS HERE ---
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v2} \
    --base_model_path {base_model_name} \
    --test_set_path {golden_smoke_test_path} \
    --breakdown # Use the corrected argument name

print("\n\n✅ ✅ ✅ GOLDEN SMOKE TEST COMPLETE. ✅ ✅ ✅")
print("Check the breakdown table above. If it is populated correctly, you are clear to run the full targeted evaluation.")

--- Preparing a 'Golden' smoke test for the breakdown-by-primitive analysis ---

[1/2] Loading full test set from data/migration_test_sandbox/test_set_v2_flat.json to find representative samples...


FileNotFoundError: [Errno 2] No such file or directory: 'data/migration_test_sandbox/test_set_v2_flat.json'

In [ ]:
# --- TARGETED RUN: V2 MODEL EVALUATION & BREAKDOWN ONLY (Definitive Version) ---

print("--- Preparing for the targeted V2 model evaluation run ---")

# Define all necessary paths
diversity_test_set = 'data/migration_test_sandbox/test_set_v2_flat.json'
model_path_v2 = "models/v2_flat_model/final_checkpoint"
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Define the path for the final output report
results_dir = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports"
report_path_3 = f"{results_dir}/report_3_v2_on_diversity_final.txt"

# Create the results directory on Google Drive
!mkdir -p {results_dir}
print(f"✅ Final evaluation report will be saved to: {report_path_3}")


# --- RUN 3 (Targeted): V2 Model on DIVERSITY Test Set (Proving Superiority & Breakdown) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 3/3 (Targeted Run): V2 Model on DIVERSITY v2 Test Set ---\n" + "="*80 + "\n")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v2} \
    --base_model_path {base_model_name} \
    --test_set_path {diversity_test_set} \
    --breakdown > {report_path_3}

# Also print the final, complete report to the screen
!cat {report_path_3}

print(f"\n\n✅ ✅ ✅ FINAL EVALUATION COMPLETE. Report saved to Google Drive. ✅ ✅ ✅")

--- Preparing for the targeted V2 model evaluation run ---
✅ Final evaluation report will be saved to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports/report_3_v2_on_diversity_final.txt


--- EVALUATION 3/3 (Targeted Run): V2 Model on DIVERSITY v2 Test Set ---

2025-09-05 23:39:47.163676: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 23:39:47.182439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757115587.204750  304236 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757115587.2116

In [5]:
# --- "ALL-IN-ONE" SMOKE TEST ---
import json
import os
import random

print("--- Preparing a 'Golden' smoke test for the full evaluation gauntlet ---")

# --- Step 1: Define all paths ---
# We use the final, full v2 test set as the source for our diverse smoke samples
full_diversity_test_set = 'data/migration_test_sandbox/test_set_v2_flat.json'
golden_smoke_test_path = 'data/migration_test_sandbox/golden_smoke_test_final.json'
model_path_v1 = "models/v1_flat_model/final_checkpoint"
model_path_v2 = "models/v2_flat_model/final_checkpoint"
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# --- Step 2: Programmatically build the golden test set ---
print(f"\n[1/3] Loading full diverse test set to find representative samples...")
with open(full_diversity_test_set, 'r', encoding='utf-8') as f:
    full_data = json.load(f)

# Create a small, diverse sample set
seen_primitives = set()
golden_test_set = []
random.shuffle(full_data)
for item in full_data:
    deobfuscated_cmd = item.get('response', {}).get('deobfuscated_command')
    if deobfuscated_cmd and deobfuscated_cmd not in seen_primitives:
        golden_test_set.append(item)
        seen_primitives.add(deobfuscated_cmd)
    if len(golden_test_set) >= 5:
        break

with open(golden_smoke_test_path, 'w') as f:
    json.dump(golden_test_set, f, indent=2)
print(f"✅ Created a golden smoke test file with {len(golden_test_set)} diverse primitives.")

# --- Step 3: Execute the smoke test ---
print("\n[2/3] Executing smoke test for EVALUATION 2 (V1 on Diversity)...")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v1} \
    --base_model_path {base_model_name} \
    --test_set_path {golden_smoke_test_path}

print("\n[3/3] Executing smoke test for EVALUATION 3 (V2 on Diversity with Breakdown)...")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v2} \
    --base_model_path {base_model_name} \
    --test_set_path {golden_smoke_test_path} \
    --breakdown

print("\n\n✅ ✅ ✅ ALL-IN-ONE SMOKE TEST COMPLETE. ✅ ✅ ✅")
print("If two reports and a populated breakdown table were printed above without errors, you are clear to run the full evaluation.")

--- Preparing a 'Golden' smoke test for the full evaluation gauntlet ---

[1/3] Loading full diverse test set to find representative samples...
✅ Created a golden smoke test file with 5 diverse primitives.

[2/3] Executing smoke test for EVALUATION 2 (V1 on Diversity)...
2025-09-06 10:25:37.968451: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-06 10:25:37.986579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757154338.010320   44237 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757154338.020737   44237 cuda_blas.c

In [6]:
# --- FINAL EVALUATION GAUNTLET (EVAL 2 & 3) ---

print("--- Preparing for the final evaluation runs (2/3 and 3/3) ---")

# Define all paths
diversity_test_set = 'data/migration_test_sandbox/test_set_v2_flat.json'
model_path_v1 = "models/v1_flat_model/final_checkpoint"
model_path_v2 = "models/v2_flat_model/final_checkpoint"
base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Define paths for the output report files
results_dir = "/content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports"
report_path_2 = f"{results_dir}/report_2_v1_on_diversity_final.txt"
report_path_3 = f"{results_dir}/report_3_v2_on_diversity_final.txt"
!mkdir -p {results_dir}
print(f"✅ Evaluation reports will be saved to: {results_dir}")

# --- RUN 2: V1 Model on DIVERSITY Test Set (Revealing Brittleness) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 2/3: V1 Model on DIVERSITY v2 Test Set (Away Game) ---\n" + "="*80 + "\n")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v1} \
    --base_model_path {base_model_name} \
    --test_set_path {diversity_test_set} > {report_path_2}
!cat {report_path_2}
print(f"✅ Report for Evaluation 2/3 saved to {report_path_2}")

# --- RUN 3: V2 Model on DIVERSITY Test Set (Proving Superiority & Breakdown) ---
print("\n\n" + "="*80 + "\n--- EVALUATION 3/3: V2 Model on DIVERSITY v2 Test Set (Proving Superiority) ---\n" + "="*80 + "\n")
!python -m powershell_sentinel.evaluate \
    --model_path {model_path_v2} \
    --base_model_path {base_model_name} \
    --test_set_path {diversity_test_set} \
    --breakdown > {report_path_3}
!cat {report_path_3}
print(f"✅ Report for Evaluation 3/3 saved to {report_path_3}")

print("\n\n✅ ✅ ✅ FINAL EVALUATIONS COMPLETE. All reports are saved to Google Drive. ✅ ✅ ✅")

--- Preparing for the final evaluation runs (2/3 and 3/3) ---
✅ Evaluation reports will be saved to: /content/drive/MyDrive/PowerShell_Sentinel_Final_Deliverable/evaluation_reports


--- EVALUATION 2/3: V1 Model on DIVERSITY v2 Test Set (Away Game) ---

2025-09-06 10:32:37.450209: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-06 10:32:37.468026: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757154757.489513   46154 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757154757.496094   46154 cuda_blas.cc:1407] Unable to 